In [1]:
%pip install git+https://github.com/Altanali/amazon-braket-sdk-python.git@only_validation

  Cloning https://github.com/Altanali/amazon-braket-sdk-python.git (to revision only_validation) to /private/var/folders/17/wqjdf0nn4dj11p1h8yrx44100000gr/T/pip-req-build-djnhudqx
  Running command git clone --filter=blob:none --quiet https://github.com/Altanali/amazon-braket-sdk-python.git /private/var/folders/17/wqjdf0nn4dj11p1h8yrx44100000gr/T/pip-req-build-djnhudqx
  Running command git checkout -b only_validation --track origin/only_validation
  Switched to a new branch 'only_validation'
  branch 'only_validation' set up to track 'origin/only_validation'.
  Resolved https://github.com/Altanali/amazon-braket-sdk-python.git to commit 4919883a979ee416f5d9f2314b613ef60e036ff0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for amazon-braket-sdk: filename=amazon_braket_sdk-1.95.1.dev0-py3-none-any.whl size=352262 sha256=419e992ff425175c0fb8534dbf0ed475ea06521307293d8d0612daa083fc0c5a


# Local emulation for verbatim circuits on Amazon Braket

State the purpose of this notebook and briefly describe what will be included in this notebook. State the scope of the device emulation which is to emulating a given verbatim circuit against a given set of device properties. State that it can be 

## Emulating verbatim circuits for AWS devices

1. Validation\
    a. Happy path\
    b. Unhappy path\

        aa. Qubits are all in the devices
        bb. Two-qubit gates are on existing edges
        cc. The one and two-qubit gates are supported native gates
    c. For all the 3 devices
2. Noisy simulation\
    a. one device?\
    b. compare noiseless, noisy and qpu results

In [ ]:
from braket.aws.aws_device import AwsDevice
from braket.circuits import Circuit

In [ ]:
aria_1 = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1")
garnet = AwsDevice("arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet")
ankaa3 = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")

garnet_emulator = garnet.emulator()
ankaa3_emulator = ankaa3.emulator()

In [ ]:

verbatim_circuit_garnet = Circuit().add_verbatim_box(
    Circuit().prx(1,0,0).cz(1,2).prx(2,3.14, 0).cz(4,5)
)

non_valid_verbatim_circuit_garnet = Circuit().add_verbatim_box(
    Circuit().prx(0,0,0).cz(0,1).prx(1,3.14, 0)
)

print(verbatim_circuit_garnet)
print(non_valid_verbatim_circuit_garnet)

In [ ]:
garnet_emulator.run(verbatim_circuit_garnet)

### Break down the local emulation: What are happening behind the scene

In [ ]:
garnet_emulator.validate(verbatim_circuit_garnet)

In [ ]:
noisy_circ =  garnet_emulator.transform(verbatim_circuit_garnet)
print(noisy_circ)

## Emulating verbatim circuits with custom device properties
1. History data of an existing AWS device
2. Custom device with fake device properties

In [32]:
from braket.emulation.local_emulator import LocalEmulator

In [33]:
# fake history data
garnet_properties_json = garnet.properties.json()

In [66]:
import json

In [70]:
with open("myjson.json", "w") as f:
    json.dump(garnet_properties_json, f)

In [71]:
with open("myjson.json", "r") as f:
    garnet_properties_json_v2 = json.load(f)

In [75]:
garnet_emulator_v3 = LocalEmulator.from_json(garnet_properties_json_v2)

In [59]:
garnet_properties_json_v2 = np.load("json_trial.npz")['myjson']
# npzfile = np.load("json_trial.npz")
# npzfile['myjson']

In [35]:
garnet_emulator_v2 = LocalEmulator.from_json(garnet_properties_json)

In [37]:
emu_task = garnet_emulator_v2.run(verbatim_circuit_garnet, shots=10)
emu_task.result()

GateModelQuantumTaskResult(task_metadata=TaskMetadata(braketSchemaHeader=BraketSchemaHeader(name='braket.task_result.task_metadata', version='1'), id='04880de1-c161-4671-8511-c2a5c7b1db7f', shots=10, deviceId='braket_dm', deviceParameters=None, createdAt=None, endedAt=None, status=None, failureReason=None), additional_metadata=AdditionalMetadata(action=Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[4] b;\nqubit[6] q;\nprx(0.0, 0.0) q[1];\n#pragma braket noise depolarizing(0.0010502178293543274) q[1]\ncz q[1], q[2];\n#pragma braket noise two_qubit_depolarizing(0.005830523881227623) q[1], q[2]\nprx(3.14, 0.0) q[2];\n#pragma braket noise depolarizing(0.0016998258112977949) q[2]\ncz q[4], q[5];\n#pragma braket noise two_qubit_depolarizing(0.00813071641604357) q[4], q[5]\nb[0] = measure q[1];\nb[1] = measure q[2];\nb[2] = measure q[4];\nb[3] = measure q[5];', inputs={}), dwaveMetadata=None, ionqMetadata=None, rigetti

In [ ]:
task = garnet.run(verbatim_circuit_garnet, shots=10)

In [39]:
task

AwsQuantumTask('id/taskArn':'arn:aws:braket:eu-north-1:545821822555:quantum-task/26b36ab5-7e57-458c-8b23-69a5e622c922')